## Реализация алгоритма шифрования RSA

### Алгоритм генерации ключей

1. Пользователь A генерирует два больших простых числа p и q, отличных друг от друга. При этом |p-q| – большое число, хотя p и q имеют приблизительно одинаковый битовый размер.

In [1]:
from sympy import randprime

def gen_simple_numbers(bits):
    """
    Функция для генерации простых чисел с заданным битовым размером.
    """
    while True:
        # функция randprime генерирует простое число с заданным битовым размером
        p = randprime(2 ** (bits - 1), 2**bits - 1)
        q = randprime(2 ** (bits - 1), 2**bits - 1)

        # проверка соответствия условию
        # |p-q| – большое число, хотя p и q имеют приблизительно одинаковый битовый размер.
        if p != q and abs(p - q) > 2 ** (bits // 2):
            return p, q

2. Держа p и q в секрете, Пользователь A вычисляет их произведение n=pq, которое называют модулем алгоритма.

In [2]:
def algorithm_module(p, q):
    """
    Функция для вычисления модуля алгоритма.
    """
    return p * q

3. Пользователь A вычисляет значение функции Эйлера для n по формуле φ(n)=(p-1)(q-1).

In [3]:
def euler_function(p, q):
    """
    Функция для вычисления значения функции Эйлера.
    """
    return (p - 1) * (q - 1)

4. Пользователь A выбирает целое число e, взаимно простое со значением функции φ(n). Это число называется экспонентой зашифрования.

In [4]:
from secrets import randbits
from math import gcd

def exponent_encryption(phi):
    """
    Функция для вычисления значения экспоненты зашифрования.
    """
    while True:
        # генерация случайного 16-битного числа с помощью функции randbits
        e = randbits(16)

        # проверка на взаимную простоту значений функции Эйлера и сгенерированного
        # случаного числа с помощью функции gcd
        if gcd(e, phi) == 1:
            return e

5. Пользователь A применяет расширенный алгоритм Евклида к паре чисел e и φ(n) и вычисляет значение d, удовлетворяющее соотношению ed≡1 mod φ(n). Это значение называется экспонентой расшифрования.

In [5]:
def gen_private_key(e, phi):
    """
    Функция для вычисления значения закрытого ключа.
    """
    # функция pow используется для возведения числа в степень по модулю
    # в основе логики лежит "метод двоичного возведения в степень"
    return pow(e, -1, phi)

6. Пара (e,n) публикуется в качестве открытого ключа пользователя A, d является закрытым ключом и держится в секрете.

### Алгоритм зашифрования

In [6]:
from base64 import b64encode, b64decode

def rsa_encryption(message, e, n):
    """
    Функция для зашифрования сообщения с использованием алгоритма RSA.
    """
    # кодирование исходного сообщения в "utf-8" и затем кодирование в base64
    bytes_data_encoded = b64encode(message.encode("utf-8"))

    # рассчет оптимального размера блока для шифрования
    block_size = (n.bit_length() - 1) // 8

    # разбиение закодированного исходного сообщения на блоки
    blocks_encoded = [
        bytes_data_encoded[i : i + block_size]
        for i in range(0, len(bytes_data_encoded), block_size)
    ]

    # преобразование каждого блока байтов в челое число
    for i in range(len(blocks_encoded)):
        blocks_encoded[i] = int.from_bytes(blocks_encoded[i], byteorder="big")

    # шифрование каждого блока исходного сообщения открытыми ключами e и n
    blocks_encrypted = [pow(block, e, n) for block in blocks_encoded]

    return blocks_encrypted

### Алгоритм расшифрования

In [7]:
def rsa_decryption(blocks_encrypted, n, d):
    """
    Функция для расшифровки сообщения с использованием алгоритма RSA.
    """
    # расшифрование каждого блока исходного сообщения закрытым ключем d и открытым ключем n
    blocks_decrypted = [pow(block, d, n) for block in blocks_encrypted]

    # преобразование расшифрованных числовых блоков обратно в байты
    # для каждого блока определяем необходимый размер в байтах и преобразуем
    # (bit_length() + 7) // 8 - формула для вычисления количества байт, необходимых для хранения числа
    blocks_decoded = [
        block.to_bytes((block.bit_length() + 7) // 8, byteorder="big")
        for block in blocks_decrypted
    ]

    # Сборка всех байтовых блоков в единую байтовую строку
    reassembled = b"".join(blocks_decoded)

    # Декодирование из base64 и преобразование в utf-8 строку
    message = b64decode(reassembled).decode("utf-8")
    
    return message

### Тело программы

In [10]:
while True:
    print("\nВыберите действие:")
    print("1. Генерация ключевых пар")
    print("2. Шифрование текста")
    print("3. Расшифрование текста")
    print("4. Выход")
    
    choice = input("> ")
    
    if choice == "1":
        # bits = int(input("\nВведите битовый размер для p, q: "))
        p, q = gen_simple_numbers(16)
        n = algorithm_module(p, q)
        phi = euler_function(p, q)
        e = exponent_encryption(phi)
        d = gen_private_key(e, phi)
        print(f"\nОткрытый ключ:\ne = {e}\nn = {n}")
        print(f"\nЗакрытый ключ:\nd = {d}\n")
    elif choice == "2":
        message_original = input("\nВведите исходное сообщение: ")
        e = int(input("\nВведите открытый ключ e: "))
        n = int(input("\nВведите открытый ключ n: "))
        blocks_encrypted = rsa_encryption(message_original, e, n)
        print(f"\nЗашифрованное сообщение:\n{' '.join(map(str, blocks_encrypted))}")
    elif choice == "3":
        blocks_encrypted = list(map(int, input("\nВведите зашифрованное сообщение: ").split()))
        n = int(input("\nВведите открытый ключ n: "))
        d = int(input("\nВведите закрытый ключ d: "))
        blocks_decrypted = rsa_decryption(blocks_encrypted, n, d)
        print(f"\nРасшифрованное сообщение:\n{blocks_decrypted}")
    elif choice == "4":
        print("\nВыход из программы.")
        break
    else:
        print("\nНеверный ввод. Попробуйте снова.")


Выберите действие:
1. Генерация ключевых пар
2. Шифрование текста
3. Расшифрование текста
4. Выход


>  3

Введите зашифрованное сообщение:  782807357 2275514703 1284166791 574946770 2244577172 188200659 98109902 2050191370 298922358 1174015503 2405703483 103829573 2229403252 2275514703 2064841360 1457144120 1501082683 1426957373 1284166791 2931496822 1262761751 1646379569 871595177 1500687081 2360287916 2477096232 2836195676 1071682518 57766173 2275514703 452101542 821284826 519206889 2243296851

Введите открытый ключ n:  2940774847

Введите закрытый ключ d:  747117385



Расшифрованное сообщение:
Реализация процесса зашифрования текста

Выберите действие:
1. Генерация ключевых пар
2. Шифрование текста
3. Расшифрование текста
4. Выход


>  4



Выход из программы.


## Реализация атаки Ферма на криптосистему RSA

In [11]:
import math
from math import isqrt
from sympy import mod_inverse

def fermat_factorization(n):
    """Факторизация n методом Ферма (если p и q близки)"""
    a = isqrt(n) + 1  # округляем вверх
    b_squared = a * a - n
    while not math.isqrt(b_squared) ** 2 == b_squared:
        a += 1
        b_squared = a * a - n
    b = isqrt(b_squared)
    p = a + b
    q = a - b
    return p, q

def find_private_key(e, n):
    """Находит закрытый ключ d, зная e и n"""
    p, q = fermat_factorization(n)
    phi = (p - 1) * (q - 1)
    d = mod_inverse(e, phi)
    return d

# Открытый ключ
e = 35617
n = 2940774847

# Находим закрытый ключ
d = find_private_key(e, n)
print(f"Закрытый ключ d = {d}")

Закрытый ключ d = 747117385
